In [ ]:
import os

os.environ["MLFLOW_TRACKING_URI"] = ""
os.environ["MLFLOW_TRACKING_USERNAME"] = ""
os.environ["MLFLOW_TRACKING_PASSWORD"] = ""

In [2]:
import os
%pwd

'/Users/phuc.buidang/Documents/Learn/Projects/basics_ds_project/research'

In [3]:
os.chdir("../")
%pwd

'/Users/phuc.buidang/Documents/Learn/Projects/basics_ds_project'

In [ ]:
from dataclasses import dataclass
from pathlib import Path


@dataclass
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str

In [ ]:
from src.datascience.constants import *
from src.datascience.utils.common import read_yaml, create_directories, save_json

2025-04-06 14:40:40.122 | DEBUG    | ipykernel.kernelbase:execute_request:802 - {'header': {'msg_id': '8064d8e9-aac44a8455b69c00e20d9d8a_53216_54', 'msg_type': 'execute_reply', 'username': 'phuc.buidang', 'session': '8064d8e9-aac44a8455b69c00e20d9d8a', 'date': datetime.datetime(2025, 4, 6, 7, 40, 40, 122592, tzinfo=datetime.timezone.utc), 'version': '5.3'}, 'msg_id': '8064d8e9-aac44a8455b69c00e20d9d8a_53216_54', 'msg_type': 'execute_reply', 'parent_header': {'date': datetime.datetime(2025, 4, 6, 7, 40, 39, 617000, tzinfo=tzutc()), 'msg_id': '301f1c31-8346-44be-92a7-0f2ad3af0a8f', 'msg_type': 'execute_request', 'session': 'c92cbc4b-e122-493e-b0c5-e2419c895b1f', 'username': '921bbd28-62a6-495f-bf76-e99e98a192ba', 'version': '5.2'}, 'content': {'status': 'ok', 'execution_count': 5, 'user_expressions': {}, 'payload': []}, 'metadata': {'started': datetime.datetime(2025, 4, 6, 7, 40, 39, 618196, tzinfo=datetime.timezone.utc), 'dependencies_met': True, 'engine': '6ab67d15-c85c-457a-bb39-0719e

In [ ]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH,
        schema_filepath=SCHEMA_FILE_PATH,
    ):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            all_params=params,
            metric_file_name=config.metric_file_name,
            target_column=schema.name,
            mlflow_uri="",
        )
        return model_evaluation_config

2025-04-06 14:40:52.676 | DEBUG    | ipykernel.kernelbase:dispatch_shell:419 - 
*** MESSAGE TYPE:execute_request***
2025-04-06 14:40:52.677 | DEBUG    | ipykernel.kernelbase:dispatch_shell:420 -    Content: {'silent': False, 'store_history': True, 'user_expressions': {}, 'allow_stdin': True, 'stop_on_error': False, 'code': 'class ConfigurationManager:\n    def __init__(\n        self,\n        config_filepath = CONFIG_FILE_PATH,\n        params_filepath = PARAMS_FILE_PATH,\n        schema_filepath = SCHEMA_FILE_PATH):\n\n        self.config = read_yaml(config_filepath)\n        self.params = read_yaml(params_filepath)\n        self.schema = read_yaml(schema_filepath)\n\n        create_directories([self.config.artifacts_root])\n\n    def get_model_evaluation_config(self) -> ModelEvaluationConfig:\n        config=self.config.model_evaluation\n        params=self.params.ElasticNet\n        schema=self.schema.TARGET_COLUMN\n\n        create_directories([config.root_dir])\n\n        model_e

In [7]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

2025-04-06 14:40:55.148 | DEBUG    | ipykernel.kernelbase:dispatch_shell:419 - 
*** MESSAGE TYPE:execute_request***
2025-04-06 14:40:55.148 | DEBUG    | ipykernel.kernelbase:dispatch_shell:420 -    Content: {'silent': False, 'store_history': True, 'user_expressions': {}, 'allow_stdin': True, 'stop_on_error': False, 'code': 'import os\nimport pandas as pd\nfrom sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score\nfrom urllib.parse import urlparse\nimport mlflow\nimport mlflow.sklearn\nimport numpy as np\nimport joblib'}
   --->
   
2025-04-06 14:40:55.149 | DEBUG    | ipykernel.kernelbase:dispatch_shell:429 - execute_request: {'header': {'date': datetime.datetime(2025, 4, 6, 7, 40, 55, 146000, tzinfo=tzutc()), 'msg_id': 'dfe191e9-5abd-46be-b05d-85d8227662d3', 'msg_type': 'execute_request', 'session': 'c92cbc4b-e122-493e-b0c5-e2419c895b1f', 'username': '921bbd28-62a6-495f-bf76-e99e98a192ba', 'version': '5.2'}, 'msg_id': 'dfe191e9-5abd-46be-b05d-85d8227662d3', 'msg_ty

In [10]:
import mlflow.models


class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self, actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2

    def log_into_mlflow(self):

        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():

            predicted_qualities = model.predict(test_x)

            (rmse, mae, r2) = self.eval_metrics(test_y, predicted_qualities)

            # Saving metrics as local
            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("r2", r2)
            mlflow.log_metric("mae", mae)

            signature = mlflow.models.infer_signature(test_x, predicted_qualities)

            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.sklearn.log_model(
                    model,
                    "model",
                    registered_model_name="ElasticnetModel",
                    signature=signature,
                )
            else:
                mlflow.sklearn.log_model(model, "model", signature=signature)

2025-04-06 14:42:29.251 | DEBUG    | ipykernel.kernelbase:dispatch_shell:419 - 
*** MESSAGE TYPE:execute_request***
2025-04-06 14:42:29.252 | DEBUG    | ipykernel.kernelbase:dispatch_shell:420 -    Content: {'silent': False, 'store_history': True, 'user_expressions': {}, 'allow_stdin': True, 'stop_on_error': False, 'code': 'import mlflow.models\n\n\nclass ModelEvaluation:\n    def __init__(self, config: ModelEvaluationConfig):\n        self.config = config\n\n    def eval_metrics(self, actual, pred):\n        rmse = np.sqrt(mean_squared_error(actual, pred))\n        mae = mean_absolute_error(actual, pred)\n        r2 = r2_score(actual, pred)\n        return rmse, mae, r2\n\n    def log_into_mlflow(self):\n\n        test_data = pd.read_csv(self.config.test_data_path)\n        model = joblib.load(self.config.model_path)\n\n        test_x = test_data.drop([self.config.target_column], axis=1)\n        test_y = test_data[[self.config.target_column]]\n\n        mlflow.set_registry_uri(self.c

In [11]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    model_evaluation.log_into_mlflow()
except Exception as e:
    raise e

2025-04-06 14:42:31.236 | DEBUG    | ipykernel.kernelbase:dispatch_shell:419 - 
*** MESSAGE TYPE:execute_request***
2025-04-06 14:42:31.237 | DEBUG    | ipykernel.kernelbase:dispatch_shell:420 -    Content: {'silent': False, 'store_history': True, 'user_expressions': {}, 'allow_stdin': True, 'stop_on_error': False, 'code': 'try:\n    config = ConfigurationManager()\n    model_evaluation_config = config.get_model_evaluation_config()\n    model_evaluation = ModelEvaluation(config=model_evaluation_config)\n    model_evaluation.log_into_mlflow()\nexcept Exception as e:\n    raise e'}
   --->
   
2025-04-06 14:42:31.237 | DEBUG    | ipykernel.kernelbase:dispatch_shell:429 - execute_request: {'header': {'date': datetime.datetime(2025, 4, 6, 7, 42, 31, 234000, tzinfo=tzutc()), 'msg_id': 'b7105193-4f64-4dfb-8c68-2b265a829903', 'msg_type': 'execute_request', 'session': 'c92cbc4b-e122-493e-b0c5-e2419c895b1f', 'username': '921bbd28-62a6-495f-bf76-e99e98a192ba', 'version': '5.2'}, 'msg_id': 'b7105